In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits, on='title')

In [4]:
movies = movies[['movie_id','title','overview','genres', 'keywords','cast','crew']]

In [5]:
movies.dropna(inplace=True)

In [6]:
movies.duplicated().sum()

np.int64(0)

In [7]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [8]:
import ast
ast.literal_eval(movies.iloc[0].genres)

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [9]:
movies['genres'] = movies['genres'].apply(convert)

In [10]:
movies['keywords'] = movies['keywords'].apply(convert)

In [11]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [12]:
movies['cast'] = movies['cast'].apply(convert3)


In [13]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [14]:
movies['crew'] = movies['crew'].apply(fetch_director)


In [15]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [16]:
movies['genres'] = movies['genres'].apply(lambda x:[ i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[ i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[ i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[ i.replace(" ","") for i in x])

In [17]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [18]:
new_df = movies[['movie_id', 'title', 'tags']] 

In [19]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\techb\AppData\Local\Temp\ipykernel_14240\1824047427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [20]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

C:\Users\techb\AppData\Local\Temp\ipykernel_14240\1380776331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [21]:
import nltk
from nltk.stem.porter import PorterStemmer 
ps = PorterStemmer()

In [22]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)


In [23]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\techb\AppData\Local\Temp\ipykernel_14240\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [65]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=30000, stop_words='english')

In [66]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [67]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'érictessi', 'étiennefaur', 'été'],
      shape=(30000,), dtype=object)

In [68]:
from sklearn.metrics.pairwise import cosine_similarity


In [69]:
similarity = cosine_similarity(vectors)

In [70]:
sorted(list(enumerate(similarity[0])),reverse=True, key=lambda x:x[1])[1:6]

[(2405, np.float64(0.22318099025284097)),
 (1214, np.float64(0.21870444131992658)),
 (539, np.float64(0.21428571428571427)),
 (582, np.float64(0.1995716281431565)),
 (507, np.float64(0.19928166641797587))]

In [71]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distance = similarity[movie_index]
    movies_list = sorted(list(enumerate(distance)),reverse=True, key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [72]:
recommend('Spider-Man')

Spider-Man 3
Spider-Man 2
The Amazing Spider-Man 2
Arachnophobia
The Amazing Spider-Man
